In [ ]:
!pip install --upgrade pip

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
  Using cached pip-20.3.4-py2.py3-none-any.whl (1.5 MB)
  Using cached pip-20.3.3-py2.py3-none-any.whl (1.5 MB)


In [ ]:
!pip install tensorflow

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [ ]:
!pip install dlib

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [ ]:
num_images = 2054
import cv2
import os 
import dlib
import numpy as np
p = "../mmod_human_face_detector.dat"
cnn_face_detector = dlib.cnn_face_detection_model_v1(p)
from pathlib import Path
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm_notebook
import itertools
import shutil
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')#made change
mixed_precision.set_policy(policy)

In [ ]:
import tensorflow as tf
gpus=tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [ ]:
def get_input(folder):
    images = np.zeros((num_images,128,128,3), dtype = np.float32)
    imagespath = range(0,2054)
    for i in range(num_images):
        a= str(imagespath[i]).zfill(2)
        img = cv2.resize(plt.imread(folder+a+'.jpg'), (128, 128), interpolation=cv2.INTER_CUBIC)/255
        print(a)
        images[i] = img
        
    return [images]

In [ ]:
model_path = 'models/CNN50_Tom_Cruise/'
attempt = 0
foldername = model_path+"_xceptionimagenet_attempt"+str(attempt)+"/"
img_size = 128
modelname = foldername+"_xceptionimagenet_"+str(num_images)+"_"+str(img_size)
cnn_model = load_model(modelname+".h5")

In [ ]:
test_folder="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/real/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim/"
batch_input = get_input(test_folder)
batch_x = np.array(batch_input)
probr = cnn_model.predict(batch_x)

In [ ]:
test_folder="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/fake/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim/"

batch_input = get_input(test_folder)
batch_x = np.array(batch_input)
probf = cnn_model.predict(batch_x)

In [ ]:
model_path = 'models/CNN100/Tom_Cruise/tom_cruise'
attempt = 0
foldername = model_path+"_xceptionimagenet_attempt"+str(attempt)+"/"
img_size = 128
modelname = foldername+"_xceptionimagenet_"+str(num_images)+"_"+str(img_size)
cnn_model = load_model(modelname+".h5")

In [ ]:
test_folder="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/real/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim/"
batch_input = get_input(test_folder)
batch_x = np.array(batch_input)
probr1 = cnn_model.predict(batch_x)
test_folder="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/fake/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim/"
batch_input = get_input(test_folder)
batch_x = np.array(batch_input)
probf1 = cnn_model.predict(batch_x)

In [ ]:
print("cnn 50")
print("real")
f = [f for f in range(0,2054) if probr[0,f,0]<0.5]
print(len(f))
print("fake")
f = [f for f in range(0,2054) if probf[0,f,0]>=0.5]
print(len(f))

In [ ]:
print("cnn 100")
print("real")
f = [f for f in range(0,2054) if probr1[0,f,0]<0.6]
print(len(f))
print("fake")
f = [f for f in range(0,2054) if probf1[0,f,0]>=0.6]
print(len(f))

In [ ]:
for i in range(1):
    try:
        vidcap = cv2.VideoCapture('../Tom_Cruise/TOM CRUISE 2020 Deepfake VFX Breakdown Side by Side_1080p_Trim.mp4')
        success,image = vidcap.read()
        count = 0
        success = True
        frames_path="../Tom_Cruise/Completeframe/TOM_CRUISE_2020_Deepfake_VFX_Breakdown_Side_by_Side_1080p_Trim/"
        print("NOW FRAMING")
        imagespath = range(0,2241)
        kr=0
        kf=0
        prediction_folder_path='Tom_Cruise/Reconstructed/frames1/'
        Path(prediction_folder_path).mkdir(parents=True, exist_ok = True)
        real_count_real=0
        fake_count_real=0
        real_count_fake=0
        fake_count_fake=0
        print("NOW DETECTING")
        m=0
        #Perform face detection
        for it in range(0,2241):
            #try:
            a= str(imagespath[it]).zfill(4)
            image=cv2.imread(frames_path+a+'.jpg')
            
            if image is None:
                print("Could not read input image")
                continue

            faces_cnn = cnn_face_detector(image[:,:,::-1],0)
            
            if(len(faces_cnn)!=2):
                print("Total Faces =     " ,len(faces_cnn))
                cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                continue
            else:
                odd=1
                for face in faces_cnn:
                    
                    x = face.rect.left()
                    y = face.rect.top()
                    w= face.rect.right() - x
                    h= face.rect.bottom() - y 
                    y1 =y-50
                    y2= y+h+30
                    x1=x-15
                    x2=x+w+50
                    
                    if odd==1:
                        odd=0
                        result=probr1[0,kr,0]
                        kr+=1
                        if(kr==2054):
                            print(a)
                        if(result<0.5):
                            print("real")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
                            cv2.putText(image, 'REAL:'+str(result), (30, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11), thickness=2,lineType=cv2.LINE_AA) 
                            real_count_real+=1
                            
                        else:
                            print("fake")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
                            cv2.putText(image, 'FAKE:'+str(result), (30, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                            fake_count_real+=1
                        real_count= real_count_real
                        fake_count= fake_count_real
                        cv2.putText(image, 'REAL:'+str(real_count), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11),thickness=2, lineType=cv2.LINE_AA) 
                        cv2.putText(image, 'FAKE:'+str(fake_count), (30, 450), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                        
                    else:
                        odd=1
                        result=probf1[0,kf,0]
                        kf+=1
                        if(kf==2054):
                            print(a)
                        if(result<0.5):
                            print("real")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
                            cv2.putText(image, 'REAL:'+str(result), (1600, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11), thickness=2,lineType=cv2.LINE_AA) 
                            real_count_fake+=1
                            
                        else:
                            print("fake")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
                            cv2.putText(image, 'FAKE:'+str(result), (1600, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                            fake_count_fake+=1
                        real_count= real_count_fake
                        fake_count= fake_count_fake
                        cv2.putText(image, 'REAL:'+str(real_count), (1600, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11),thickness=2, lineType=cv2.LINE_AA) 
                        cv2.putText(image, 'FAKE:'+str(fake_count), (1600, 450), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                print(it)
        img_array = []
        for j in range(num_images):
            
            a= str(imagespath[j]).zfill(4)
            
            try:    
                img = cv2.imread(prediction_folder_path+a+'.jpg')
                height, width, layers = img.shape
                size = (width,height)
                img_array.append(img)
            except:
                continue
        print("NOW RECONSTRUCTING")
        save_path='Tom_Cruise/Reconstructed/rvideo/'
        Path(save_path).mkdir(parents=True, exist_ok = True)
        out = cv2.VideoWriter(save_path+'Tom_Cruise_video'+'.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        print("VIDEO COMPLETED:")
        
    except Exception as e:
        print(e)
        continue

In [ ]:
frames_path="../Tom_Cruise/Completeframe/TOM_CRUISE_2020_Deepfake_VFX_Breakdown_Side_by_Side_1080p_Trim/"
count=0        
for it in range(0,2241):
    #try:
    a= str(imagespath[it]).zfill(4)
    image=cv2.imread(frames_path+a+'.jpg')

    if image is None:
        print("Could not read input image")
        continue

    faces_cnn = cnn_face_detector(image[:,:,::-1],0)

    if(len(faces_cnn)!=2):
        print("Total Faces =     " ,len(faces_cnn))
        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
        continue
    else:
        count+=1
print(count)

In [ ]:
img_array = []
for j in range(2241):

    a= str(imagespath[j]).zfill(4)

    try:    
        img = cv2.imread(prediction_folder_path+a+'.jpg')
        height, width, layers = img.shape
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    except:
        print(a)
        continue
print("NOW RECONSTRUCTING")
save_path='Tom_Cruise/Reconstructed/rvideo/'
Path(save_path).mkdir(parents=True, exist_ok = True)
out = cv2.VideoWriter(save_path+'fake_video1'+'.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
print("VIDEO COMPLETED:")

In [ ]:
for i in range(1):
    try:
        vidcap = cv2.VideoCapture('../Tom_Cruise/TOM CRUISE 2020 Deepfake VFX Breakdown Side by Side_1080p_Trim.mp4')
        success,image = vidcap.read()
        count = 0
        success = True
        frames_path="../Tom_Cruise/Completeframe/TOM_CRUISE_2020_Deepfake_VFX_Breakdown_Side_by_Side_1080p_Trim/"
        print("NOW FRAMING")
        imagespath = range(0,10)
        kr=0
        kf=0
        prediction_folder_path='Tom_Cruise/Reconstructed/frames/'
        Path(prediction_folder_path).mkdir(parents=True, exist_ok = True)
        real_count_real=0
        fake_count_real=0
        real_count_fake=0
        fake_count_fake=0
        print("NOW DETECTING")
        m=0
        #Perform face detection
        for it in range(0,10):
            #try:
            a= str(imagespath[it]).zfill(4)
            image=cv2.imread(frames_path+a+'.jpg')
            
            if image is None:
                print("Could not read input image")
                continue

            faces_cnn = cnn_face_detector(image[:,:,::-1],0)
            
            if(len(faces_cnn)!=2):
                print("Total Faces =     " ,len(faces_cnn))
                cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                continue
            else:
                odd=1
                for face in faces_cnn:
                    
                    x = face.rect.left()
                    y = face.rect.top()
                    w= face.rect.right() - x
                    h= face.rect.bottom() - y 
                    y1 =y-50
                    y2= y+h+30
                    x1=x-15
                    x2=x+w+50
                    
                    if odd==1:
                        odd=0
                        result=probr1[0,kr,0]
                        kr+=1
                        
                    else:
                        odd=1
                        result=probf1[0,kf,0]
                        kf+=1
                        
                    #printing Fake or real on eac frame
                    if(result<0.5):
                        print("hi1")
                        cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
                        cv2.putText(image, 'REAL:'+str(result), (700, 100), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,255,0), lineType=cv2.LINE_AA) 
                        if odd ==1:
                            real_count_fake+=1
                            real_count= real_count_fake
                            fake_count= fake_count_fake
                        else:
                            real_count_real+=1
                            real_count= real_count_real
                            fake_count= fake_count_real
                        cv2.putText(image, 'REAL:'+str(real_count), (700, 500), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,255, 0), lineType=cv2.LINE_AA) 
                        cv2.putText(image, 'FAKE:'+str(fake_count), (700, 550), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), lineType=cv2.LINE_AA) 
                        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                    else:
                        print("hi2")
                        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
                        cv2.putText(image, 'FAKE:'+str(result), (100, 100), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0 ,255),lineType=cv2.LINE_AA)
                        if odd ==1:
                            fake_count_fake+=1
                            real_count= real_count_fake
                            fake_count= fake_count_fake
                        else:
                            fake_count_real+=1
                            real_count= real_count_real
                            fake_count= fake_count_real
                        cv2.putText(image, 'REAL:'+str(real_count), (100, 500), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,255, 0), lineType=cv2.LINE_AA)
                        cv2.putText(image, 'FAKE:'+str(fake_count), (100, 550), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), lineType=cv2.LINE_AA) 
                        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                print(it)
        img_array = []
        for j in range(num_images):
            
            a= str(imagespath[j]).zfill(4)
            
            try:    
                img = cv2.imread(prediction_folder_path+a+'.jpg')
                height, width, layers = img.shape
                height, width, layers = img.shape
                size = (width,height)
                img_array.append(img)
            except:
                continue
        print("NOW RECONSTRUCTING")
        save_path='Tom_Cruise/Reconstructed/rvideo/'
        Path(save_path).mkdir(parents=True, exist_ok = True)
        out = cv2.VideoWriter(save_path+'fake_video'+'.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        print("VIDEO COMPLETED:")
        
    except Exception as e:
        print(e)
        continue